# Maximum Entropy Matching Distribution Function

This notebook can construct the distribution function which maximizes the kinetic entropy subject to the constraints of matching the energy density $\epsilon$ and the shear stress $\pi^{\mu\nu}$.

The entropy density current is expressed

$$s^{\mu}(x) = - \sum_h \frac{g_h}{(2\pi)^3}\int \frac{d^3p}{p_0} p^{\mu} \phi[f_h],$$

where $h$ runs over all hadronic species, $g_h$ is the degeneracy of species $h$, and $f_h(x;p)$ is the distribution function of particles of species $h$.

The function $\phi[f_h]$ defined by

$$ \phi[f] \equiv f \ln (f) - \frac{1 + \theta f}{\theta} \ln(1 + \theta f)$$

defines the quantum statistical nature of the particles, with $\theta = 1$, $0$ or $-1$ for bosons, Maxwell-Boltzmann particles, and fermions respectively.

In [1]:
%load_ext Cython
from scipy import integrate
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline
from scipy import optimize
from numpy import linalg as LA
from mpmath import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dill
import pandas as pd
import time
sns.set()
sns.set_style('whitegrid')
sns.set_context('talk')

In [2]:
#These triple series expressions for epsilon, pi_1 and pi_2 
#are still a little slow for phenomenology
#alternatively, the integral expression for the generating 
#function could be used

hbar = 0.197
#hbar = 1. #temporary

#sum_max = 10
sum_max = 10

method_sum='alternating'
#method_sum='shanks'
#method_sum='direct'

gamma = memoize(gamma) #speed up mpmath

#the angular function
def angular(nx, ny, nz):
    ns = nx+ny+nz
    a = (1+(-1)**(2*nx))*(1+(-1)**(2*ny))*(1+(-1)**(2*nz))/4.
    b = gamma((1.+ (2.*nx))/2.) * gamma((1.+ (2.*ny))/2.) * gamma((1.+ (2.*nz))/2.) / gamma((3.+(2.*ns))/2.)
    return mpmathify(a*b)

angular = memoize(angular) #speed up mpmath

def sum_e_fct(beta, g1, g2):
    beta = mpmathify(beta)
    g1 = mpmathify(g1)
    g2 = mpmathify(g2)
    g3 = -(g1+g2)
    def e_fct(nx, ny, nz):
        nx = mpmathify(nx)
        ny = mpmathify(ny)
        nz = mpmathify(nz)
            
        ns = nx + ny + nz
        a = angular(nx, ny, nz) * gamma(3+ns) / (fac(nx)*fac(ny)*fac(nz))
        b = ((-g1/beta)**nx) * ((-g2/beta)**ny) * ((-g3/beta)**nz)
        c = (3+ns)
        pre_fac = 1./ ((2.*np.pi*hbar)**3.) / (beta ** 4.)
        pre_fac = mpmathify(pre_fac)
        return pre_fac*a*b*c
    return float(nsum( e_fct, [0, sum_max], [0, sum_max], [0, sum_max], method=method_sum ))
        
def sum_pi_1_fct(beta, g1, g2):
    beta = mpmathify(beta)
    g1 = mpmathify(g1)
    g2 = mpmathify(g2)
    g3 = -(g1+g2)
    def pi_1_fct(nx, ny, nz):
        nx = mpmathify(nx)
        ny = mpmathify(ny)
        nz = mpmathify(nz)
            
        ns = nx + ny + nz
        a = (-1)**(ns) * angular(nx, ny, nz) * gamma(3+ns) / (fac(nx) * fac(ny) * fac(nz))
        b = ((g1/beta)**nx) * ((g2/beta)**ny) * ((g3/beta)**nz)
        c = ( (3.+ns)/3. + (nx*beta/g1) )
        pre_fac = -1./ ((2.*np.pi*hbar)**3.) / (beta ** 4.)
        pre_fac = mpmathify(pre_fac)
        return pre_fac*a*b*c 
    return float(nsum( pi_1_fct, [0, sum_max], [0, sum_max], [0, sum_max], method=method_sum ))

def sum_pi_2_fct(beta, g1, g2):
    beta = mpmathify(beta)
    g1 = mpmathify(g1)
    g2 = mpmathify(g2)
    g3 = -(g1+g2)
    def pi_2_fct(nx, ny, nz):
        nx = mpmathify(nx)
        ny = mpmathify(ny)
        nz = mpmathify(nz)
            
        ns = nx + ny + nz
        a = (-1)**(ns) * angular(nx, ny, nz) * gamma(3+ns) / (fac(nx) * fac(ny) * fac(nz))
        b = ((g1/beta)**nx) * ((g2/beta)**ny) * ((g3/beta)**nz)
        c = ( (3.+ns)/3. + (ny*beta/g2) )
        pre_fac = -1./ ((2.*np.pi*hbar)**3.) / (beta ** 4.)
        pre_fac = mpmathify(pre_fac)
        return pre_fac*a*b*c 
    return float(nsum( pi_2_fct, [0, sum_max], [0, sum_max], [0, sum_max], method=method_sum ))

In [3]:
#method_root='anderson'

#this root solving routine is still slow,
#but can probably be sped up with a c backend 
#and some algebraic re-arranging of the various terms
#Sage has a cython mpmath backend (https://www.sagemath.org)

def root_solve_beta_gamma(e0, pi_1, pi_2, guess, tol=1e-4, maxiter = int(1e5)):    
    
    def my_fun(x):
        beta = x[0]
        g1 = x[1]
        g2 = x[2]
        
        t1 = time.time()
        e_val = sum_e_fct(beta, g1, g2)
        t2 = time.time()
        #print("e sum evaluation: "+str(t2-t1)+" sec")
        t1 = time.time()
        pi_1_val = sum_pi_1_fct(beta, g1, g2)
        t2 = time.time()
        #print("pi_1 sum evaluation: "+str(t2-t1)+" sec")
        pi_2_val = sum_pi_2_fct(beta, g1, g2)
        #print("beta : "+str(beta)+", g1 : "+str(g1)+", g2 : "+str(g2))
        #print("e : "+str(e_val)+", pi_1 : "+str(pi_1_val)+", pi_2 : "+str(pi_2_val))
        return [e_val - e0, pi_1_val - pi_1, pi_2_val - pi_2]
    
    def print_warning(e0, pi_1, pi_2):
        print("WARNING - NO ROOT FOUND: e0 = " + str(e0) + " ; pi_1 = " + str(pi_1))
    
    solution = optimize.root(my_fun, guess, tol=tol)
    #solution = optimize.root(my_fun, guess, method=method_root, tol=tol, options={'maxiter':maxiter})
    
    if (solution.success == False):
        print_warning(e0, pi_1, pi_2)
        
    beta_soln = solution.x[0]
    g1_soln = solution.x[1]
    g2_soln = solution.x[2]
    
    return beta_soln, g1_soln, g2_soln, solution

In [4]:
#Checks that the functions are working correctly
t1 = time.time()
e = sum_e_fct(beta=1., g1=0.1, g2=0.2)
t2 = time.time()
pi_1 = sum_pi_1_fct(beta=1., g1=0.1, g2=0.2)
t3 = time.time()
pi_2 = sum_pi_2_fct(beta=1., g1=0.1, g2=0.2)
print("e = " + str(e) + " ?=? 49.772")
print("pi_1 = " + str(pi_1) + " ?=? -4.035")
print("pi_2 = " + str(pi_2) + " ?=? -5.5907")
print("e sum eval'd in " + str(t2-t1) + " sec")
print("pi_1 sum eval'd in " + str(t3-t2) + " sec")

e = 49.77232274243643 ?=? 49.772
pi_1 = -4.035753999037346 ?=? -4.035
pi_2 = -5.5907058966960825 ?=? -5.5907
e sum eval'd in 0.3910179138183594 sec
pi_1 sum eval'd in 0.21982789039611816 sec


In [5]:
#Check the root solving method
guess = [0.5, -0.1, -0.1]
root_solve_beta_gamma(e, pi_1, pi_2, guess, tol=1e-7, maxiter = int(1e5))

(1.0000000000073683,
 0.10000000003333051,
 0.19999999996688783,
     fjac: array([[-9.82519014e-01,  5.17628790e-02,  1.78821116e-01],
        [-5.24995834e-02, -9.98620758e-01,  6.13162981e-04],
        [-1.78606218e-01,  8.78558983e-03, -9.83881412e-01]])
      fun: array([-1.88182980e-09, -3.65245612e-10,  7.34086569e-10])
  message: 'The solution converged.'
     nfev: 28
      qtf: array([-1.19752542e-06, -2.83163770e-07,  2.37646739e-07])
        r: array([274.91524267, -60.80136053, -59.15909492,  35.7358067 ,
         21.9813921 ,  11.73905269])
   status: 1
  success: True
        x: array([1. , 0.1, 0.2]))

In [6]:
def find_shear_eig(pi_ij):
    """pi_ij should be the shear stress tensor in the LRF"""
    w, v = LA.eig(pi_ij)
    return w, v

In [8]:
#this function solves for all the lagrange multipliers
#and returns them
def find_lagrange_multipliers(e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz):

    pi_zz = -(pi_xx + pi_yy) #traceless

    #symmetric matrix
    pi_ij = np.array( [[pi_xx, pi_xy, pi_xz],[pi_xy, pi_yy, pi_yz],[pi_xz, pi_yz, pi_zz]] )

    w, v = find_shear_eig(pi_ij) #w are eig vals, v is matrix of eig vectors

    pi_1, pi_2 = w[0], w[1] #first two eig vals

    #now root solve to find lagrange multipliers
    beta_soln, g1_soln, g2_soln, info = root_solve_beta_gamma(e, pi_1, pi_2, guess, tol=1e-7, maxiter = int(1e5))

    g3_soln = -(g1_soln + g2_soln)
    #construct the diagonal gamma matrix
    gamma_ij_D = np.array( [[g1_soln, 0., 0.],[0., g2_soln, 0.],[0., 0., g3_soln]] )
    #now rotate gamma out of the eigenvector basis
    gamma_ij = (v.T)@(gamma_ij_D)@(v)
    
    return beta_soln, gamma_ij, info

In [9]:
e = 50.
pi_xx = 5.
pi_yy = 5.
pi_xy = 1.
pi_xz = 2.
pi_yz = 2.

find_lagrange_multipliers(e, pi_xx, pi_xy, pi_xz, pi_yy, pi_yz)

(1.0645309313584952, array([[-0.23584837, -0.06160451,  0.07566025],
        [-0.06160451,  0.11263935, -0.44098026],
        [ 0.07566025, -0.44098026,  0.12320901]]),     fjac: array([[-0.97192801,  0.20402912, -0.11716679],
        [ 0.08211396,  0.76083516,  0.64372911],
        [-0.22048409, -0.61603732,  0.75623064]])
      fun: array([-1.36609017e-08,  8.00658206e-09,  2.75050205e-10])
  message: 'The solution converged.'
     nfev: 30
      qtf: array([-1.33188422e-06, -3.65215530e-07, -4.05585651e-08])
        r: array([245.04663241, -77.16802481,  34.98087539, -29.02032542,
        -31.14533073, -27.81755229])
   status: 1
  success: True
        x: array([ 1.06453093,  0.57063282, -0.32421479]))